# Google search try

In [19]:
from googlesearch import search
import re
import pandas as pd

In [20]:
companies = pd.read_csv('../data/processed/companies.csv')

In [21]:
companies = companies['name'].tolist()

In [22]:
# to search
found = set()

for idx,company in enumerate(companies):
    for j in search(company, tld="com"):
        print(idx)
        if re.search('wikipedia',j):
            found.add(company)
            break

KeyboardInterrupt: 

In [ ]:
search

In [ ]:
import bs4 as bs
import urllib

comp = 'The Bank of East Asia Ltd'
url = 'https://en.wikipedia.org/wiki/'
print(urllib.parse.urlencode(comp))
# source = urllib.request.urlopen(url).read()

In [ ]:
soup = bs.BeautifulSoup(source,'html')

In [ ]:
for tag in soup.find_all('td'):
    if tag.get('class') == ['infobox-data', 'adr']:
        print(tag.text)

# Nominatim

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd 
import os
import sys
import ssl
import numpy as np


sys.path.append('../')

In [2]:
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [3]:
geolocator = Nominatim(user_agent="my-application3", ssl_context=ssl_context, timeout=None)

In [4]:
# Random test
location = geolocator.geocode("175 5th Avenue NYC, United states")
location.latitude 

40.741059199999995

In [5]:
address = pd.read_parquet('../data/raw/addresses.parquet.gzip')
country_codes  = pd.read_excel('../data/raw/Countries.xlsx')

In [6]:
address['completed'] = np.nan 

In [7]:
address['city'] = address['city_state_postal'].str.split(',').str[0]

In [8]:
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Aachen'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Artarmon'), 'country'] = 'AUS'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Ann Arbor'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Bangalore'), 'country'] = 'IND'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Blindern'), 'country'] = 'NOR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Boulder'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Cambridge'), 'country'] = 'GBR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Lausanne'), 'country'] = 'CHE'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Milwaukee'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Munich'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'New York'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Norwich'), 'country'] = 'GBR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Oberhaching'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Paris'), 'country'] = 'FRA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Philadelphia'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Snoqualmie'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Southborough'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Stockholm 111 37'), 'country'] = 'SWE'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Towson'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Waltham'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Zurich 8002'), 'country'] = 'CHE'

# Adding the only other company with location_street1
address.loc[(address['name'] == 'Fennoa Oy'), 'city'] = 'Alavus'
address.loc[(address['name'] == 'Fennoa Oy'), 'country'] = 'FIN'
address.loc[(address['name'] == 'Fennoa Oy'), 'city_state_postal'] = 'Alavus, 63300'
address.loc[(address['name'] == 'Fennoa Oy'), 'location_street1'] = 'Mikko Kalliovaara Rinteentie 136'

In [9]:
address.loc[(address.country == 'NAM'), 'country'] = None

In [10]:
address = address.merge(country_codes, how='left', left_on='country', right_on='ISO-alpha3 Code')

In [11]:
# Have to take care of this when finding the location
# address.loc[(~address['location_street1'].isnull())&(address.city_state_postal.isnull())] 

In [12]:
address.drop(columns=['No','M49 Code','Region 1', 'Region 2', 'country'], inplace=True)
address.rename(columns={'Country or Area':'country'}, inplace=True)

In [13]:
address.head(2)

,id,name,city_state_postal,location_street1,completed,city,country,ISO-alpha3 Code,Continent
0,50472,"Infinity Pharmaceuticals, Inc.","Cambridge, MA 02138",1100 Massachusetts Avenue,NaN,Cambridge,United States of America,USA,North America
1,282940,"Bozhon Precision Industry Technology Co., Ltd.","Suzhou, Jiangsu 215222",666 Huxin West Road,NaN,Suzhou,China,CHN,Asia


### Getting the latitude and longitude from Nominatim

In [14]:
def api_call(search_str):
    """
    This function takes a search string and returns a location object obtained from the Nomnatim API.

    Parameters:
    search_str (str): The search string to be used in the API call.

    Returns:
    location (object): The location object returned from the API call. 
    """
    try:
        location = geolocator.geocode(search_str)
        return location
    except:
        return None



def new_search_str(old_search_str, city_state_postal, location_street1, country):
    """
    This function modifies the search string to be used in the API call. If the search string is None, it creates one from the 
    city_state_postal, location_street1, and country columns. If the search string is not None, it modifies the search string.

    Parameters:
    old_search_str (str): The search string to be modified.
    city_state_postal (str): The city, state, and postal code of the company.
    location_street1 (str): The street address of the company.
    country (str): The country of the company.

    Returns:
    search_str (str): The modified search string.

    """

    if old_search_str is None:

        # we create a new search string

        if country is None:
            search_str = None
        
        else:
            if (city_state_postal is None) & (location_street1 is None):
                search_str = country
            
            if (city_state_postal is None) & (location_street1 is not None):
                search_str = location_street1 + ', ' + country
            
            if (city_state_postal is not None) & (location_street1 is None):
                search_str = city_state_postal + ', ' + country

            if (city_state_postal is not None) & (location_street1 is not None):
                search_str = location_street1 + ', ' +  city_state_postal + ', ' + country

    else:

        # If the old search string doesnt give proper results, we modify it
        # we remove the first word from the search string and try again

        search_str = old_search_str[old_search_str.find(" ")+1:]

    return search_str


def get_location(row):
    """
    This function takes a row from the address dataframe and returns a location object obtained from the Nomnatim API.

    Parameters:
    row (series): A row from the address dataframe.

    Returns:
    location (object): The location object returned from the API call. 
    """
    old_search_str = None
    x = None 

    while x is None:

        search_str = new_search_str(old_search_str, rows['city_state_postal'], rows['location_street1'], rows['country'])
        x = api_call(search_str)
        
        if x is None:
            old_search_str = search_str
            completed = 0
            continue

        if x is not None:
 
            completed = 1
            break

        if search_str == old_search_str:
            completed = 1
            break

    return completed, x

In [25]:
address_new = address.loc[(address['completed'] != 1)]

for id, rows in address_new.iterrows():

    print(id)

    if rows['completed'] != 1:

        old_search_str = None
        x = None 

        while x is None:

            search_str = new_search_str(old_search_str, rows['city_state_postal'], rows['location_street1'], rows['country'])
            x = api_call(search_str)
            
            if x is None:
                old_search_str = search_str
                address.loc[id,'completed'] = 0
                continue

            if x is not None:
                lat = x.latitude
                log = x.longitude
                address.loc[id,'lat'] = lat
                address.loc[id,'log'] = log
                address.loc[id,'completed'] = 1
                break

            if search_str == old_search_str:
                address.loc[id,'completed'] = 1
                break
            # i = 1
            # while i < len(rows['city_state_postal'].split(',')) :
            #     search_str = ','.join(rows['city_state_postal'].split(',')[0:i])+", " + rows['country']
            #     print(search_str)
            #     x = geolocator.geocode(search_str)
            #     if x is None:
            #         i += 1
            #         continue
            #     if x is not None:
            #         lat = x.latitude
            #         log = x.longitude
            #         address.loc[id,'lat'] = lat
            #         address.loc[id,'log'] = log
            #         address.loc[id,'completed'] = 1
            #         break
    
    if id % 1000 == 0:
        address.to_parquet('../data/processed/addresses1.parquet.gzip', compression='gzip')
        print('saved')

1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200


In [24]:
address = pd.read_parquet('../data/processed/addresses1.parquet.gzip')
address.loc[(address.location_street1.isnull())] 

,id,name,city_state_postal,location_street1,completed,city,country,ISO-alpha3 Code,Continent,lat,log
2149,170181,amaysim Australia Ltd.,"Wollstonecraft, New South Wales (NSW) 2065",None,NaN,Wollstonecraft,Australia,AUS,Oceania,NaN,NaN
4276,15108,WebMD Health Corp.,"New York, NY",None,NaN,New York,United States of America,USA,North America,NaN,NaN
4958,188189,"Viggle Media, Inc.","New York, NY",None,NaN,New York,United States of America,USA,North America,NaN,NaN
6634,10400,A.C.N. 004 410 833 Ltd.,"Sydney, New South Wales (NSW) 2000",None,NaN,Sydney,Australia,AUS,Oceania,NaN,NaN
6761,141488,Did Corp.,None,None,NaN,None,Republic of Korea,KOR,Asia,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
195912,298187,Mey-Ram Electricity Infrastrutures Ltd.,None,None,NaN,None,Israel,ISR,Asia,NaN,NaN
195916,307471,"KTC ozora, Inc.",None,None,NaN,None,Japan,JPN,Asia,NaN,NaN
195920,371047,Zeneca Pharmaceuticals,Delaware,None,NaN,Delaware,United States of America,USA,North America,NaN,NaN
195923,310366,Texas Health Resources,None,None,NaN,None,None,None,None,NaN,NaN


In [ ]:
address = pd.read_parquet('../data/processed/addresses.parquet.gzip')
address

In [ ]:
address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')

In [ ]:
address.country.isna().sum()

In [ ]:
country_codes 

In [ ]:
uni = address.groupby('city').agg(
    unique_country = pd.NamedAgg('country',lambda x: x.unique().tolist())
).reset_index()

In [ ]:
mult = uni.loc[uni['unique_country'].apply(lambda x: len(x)) > 1]

In [ ]:
mult_city = mult.explode('unique_country')

In [ ]:
address = mult_city.merge(country_codes, left_on='unique_country', right_on='Alpha-3 code', how='left')

In [ ]:
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Aachen'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Artarmon'), 'Country'] = 'Australia'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Ann Arbor'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Bangalore'), 'Country'] = 'India'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Boulder'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Cambridge'), 'Country'] = 'United Kingdom'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Lausanne'), 'Country'] = 'Switzerland'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Milwaukee'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Munich'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'New York'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Norwich'), 'Country'] = 'United Kingdom'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Oberhaching'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Paris'), 'Country'] = 'France'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Philadelphia'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Snoqualmie'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Southborough'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Stockholm 111 37'), 'Country'] = 'Sweden'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Towson'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Waltham'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Zurich 8002'), 'Country'] = 'Switzerland'

In [ ]:
address

In [ ]:
for id, row in address.iterrows():
    print(id)
    mask = (address['city'] == row['city'])&(address['country'] == row['unique_country'])

    x = geolocator.geocode(row['city']+ ", "+ row['Country'])
    
    if x is not None:

        lat = x.latitude
        log = x.longitude

        address.loc[mask,'lat'] = lat
        address.loc[mask,'log'] = log

    if x is None:

        address.loc[mask, 'lat'] = None
        address.loc[mask, 'log'] = None
    
    if id % 100 == 0:
        address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')
        print('saved')
    

In [ ]:
address.to_csv('../data/processed/addresses.csv')